**Forecasting Tookit**

The Forecasting Tookit is a tool for efficiently building forecasts.  It includes a collection of notebooks (exploratory data analysis, modeling, and inference) and an orchestration layer for iterating on multiple projects.

To start, run each cell one at a time, and follow the directions from the cell output.  You have the option to either use a git integration to pull from the [Emerging Solutions Toolbox](https://github.com/Snowflake-Labs/emerging-solutions-toolbox) or to upload a zipped copy of it to the created stage.

In [ ]:
# Deploys the solution.  Creates a database if it doesn't yet
# exist, and adds supporting schemas and stages, a default warehouse, and
# will add a git repository to automatically load files.
# Will check for needed permissions and will notify if missing.
# *Note* - the role running the notebook will own the objects.

# Import python packages
import os
import zipfile

import pandas as pd
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

session = get_active_session()

# Solution constants
SOLUTION_NAME = "FORECAST_MODEL_BUILDER"
TOOLBOX_FOLDER_NAME = "framework-forecast-model-builder"
DEPLOYMENT_STAGE = "NOTEBOOK_TEMPLATES"

# Permission variables
can_create_db = False
can_create_wh = False
can_create_integration = False

# Deployment variables
database_deployed = False
warehouse_deployed = False
files_deployed = False
zip_deployed = False
git_repository_deployed = False
confirm_message_sent = False


# Checks the permissions of the current role
def check_permissions(session):
    global can_create_db
    global can_create_wh
    global can_create_integration

    # Checks permissions of current role
    current_role_sql = """SELECT CURRENT_ROLE()"""

    current_role = session.sql(current_role_sql).collect()[0][0]

    admin_role_list = ["ACCOUNTADMIN", "SYSADMIN"]

    if current_role not in admin_role_list:
        grants_sql = """SHOW GRANTS ON ACCOUNT"""
        grants_df = session.sql(grants_sql)

        create_db_df = grants_df.filter(
            (col('"privilege"') == "CREATE DATABASE")
            & (col('"grantee_name"') == current_role)
        )

        if create_db_df.count() > 0:
            can_create_db = True

        create_wh_df = grants_df.filter(
            (col('"privilege"') == "CREATE WAREHOUSE")
            & (col('"grantee_name"') == current_role)
        )

        if create_wh_df.count() > 0:
            can_create_wh = True

        create_integration_df = grants_df.filter(
            (col('"privilege"') == "CREATE INTEGRATION")
            & (col('"grantee_name"') == current_role)
        )

        if create_integration_df.count() > 0:
            can_create_integration = True

    else:
        can_create_db = True
        can_create_wh = True
        can_create_integration = True


# Deploys the database and supporting objects
def deploy_database(session):
    global database_deployed

    # Check if database exists
    db_check_sql = f"""SHOW DATABASES LIKE '{SOLUTION_NAME}'"""
    db_check_df = session.sql(db_check_sql)

    if db_check_df.count() == 0 and can_create_db:
        try:
            # Create a database for the toolkit
            create_db_sql = f"""CREATE DATABASE IF NOT EXISTS {SOLUTION_NAME}
            COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_db_sql).collect()
        except Exception as e:
            st.warning(
                f"{SOLUTION_NAME} database not available and could not be created.  Please change your role or reach out to an admin.  Error: "
                + str(e),
                icon="⚠️",
            )
        finally:
            # Remove the public schema (only during initial deployment)
            remove_public_schema_sql = (
                f"""DROP SCHEMA IF EXISTS {SOLUTION_NAME}.PUBLIC"""
            )

            session.sql(remove_public_schema_sql).collect()

            # Create a schema for the toolkit itself (other schemas will be created for each project)
            create_base_schema_sql = f"""CREATE SCHEMA IF NOT EXISTS {SOLUTION_NAME}.BASE
        COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_base_schema_sql).collect()

            # Create a stage for the notebook templates used for creating new projects
            create_notebook_stage_sql = f"""CREATE STAGE IF NOT EXISTS {SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}
        DIRECTORY = (ENABLE = TRUE)
        COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_notebook_stage_sql).collect()

            database_deployed = True

    elif db_check_df.count() == 0 and not can_create_db:
        st.warning(
            f"{SOLUTION_NAME} database not available and the current role does not have the CREATE DATBASE permission.  Please change your role or reach out to an admin.",
            icon="⚠️",
        )

    elif db_check_df.count() == 1:
        database_deployed = True


# Deploys the warehouse
def deploy_warehouse(session):
    global warehouse_deployed

    # Check if warehouse exists
    wh_check_sql = f"""SHOW WAREHOUSES LIKE '{SOLUTION_NAME}_WH'"""
    wh_check_df = session.sql(wh_check_sql)

    if wh_check_df.count() == 0 and can_create_wh:
        # Create a warehouse for the toolkit
        create_wh_sql = f"""CREATE WAREHOUSE IF NOT EXISTS {SOLUTION_NAME}_WH
    WITH WAREHOUSE_SIZE = 'XSMALL'
    WAREHOUSE_TYPE = 'STANDARD'
    AUTO_SUSPEND = 10
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
    COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

        session.sql(create_wh_sql).collect()

        warehouse_deployed = True

    elif wh_check_df.count() == 0 and not can_create_wh:
        st.info(
            f"{SOLUTION_NAME}_WH warehouse not available, either use your own or rerun with a role with the CREATE WAREHOUSE permission.",
            icon="ℹ️",
        )

    elif wh_check_df.count() == 1:
        warehouse_deployed = True


# Checks if files in the stage IS missing
def check_stage(session):
    global files_deployed

    # Check if files are missing
    files_check_sql = f"""LS @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/"""
    if session.sql(files_check_sql).count() > 0:
        files_deployed = True


# Deploys the git repository
def deploy_api_integration(session):
    if database_deployed and not files_deployed:
        check_git_repository(session)
        check_for_zip(session)

        if git_repository_deployed:
            add_files_from_git(session)

        elif zip_deployed:
            add_files_from_zip(session)

        else:
            st.info(
                f"If you do not want to use git, you can manually upload the zip file from the repository to {SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE} and rerun this cell.",
                icon="ℹ️",
            )

            # Load git API integrations
            api_integrations_sql = "SHOW API INTEGRATIONS"
            api_integrations_df = session.sql(api_integrations_sql)

            if api_integrations_df.count() == 0:
                if can_create_integration:
                    api_name = st.empty()
                    api_button = st.empty()

                    api_integration_name = api_name.text_input(
                        "Name your API Integration",
                        key="api_name",
                        value="SNOWFLAKE_LABS_GIT_API_INTEGRATION",
                    )
                    if api_button.button("Create Integration", key="create_int_btn"):
                        api_integration_create_sql = f"""CREATE OR REPLACE API INTEGRATION {api_integration_name}
        API_PROVIDER = git_https_api
        API_ALLOWED_PREFIXES = ('https://github.com/Snowflake-Labs/')
        ENABLED = TRUE;"""
                        session.sql(api_integration_create_sql).collect()
                        api_name.empty()
                        api_button.empty()
                        deploy_git(session)

                else:
                    st.warning(
                        "There are no API integrations and the current role does not have permission to create one or contact your admin or manually stage the files.",
                        icon="⚠️",
                    )
            else:
                deploy_git(session)
    elif database_deployed and files_deployed:
        # Check if git is added for status
        check_git_repository(session)
        check_for_zip(session)


# Adds a git repository to the database
def deploy_git(session):
    global files_deployed
    global confirm_message_sent
    global git_repository_deployed

    # Load git API integrations
    api_integrations_sql = "SHOW API INTEGRATIONS"
    api_integrations_df = session.sql(api_integrations_sql)

    git_integrations = []

    for row in api_integrations_df.collect():
        api_integration_description_sql = (
            f"""DESCRIBE API INTEGRATION \"{row["name"]}\""""
        )
        api_integration_description_df = session.sql(
            api_integration_description_sql
        ).filter(col('"property_value"') == "GIT_HTTPS_API")

        if api_integration_description_df.count() > 0:
            git_integrations.append(row["name"])

    # Create repo if not exists
    api_select = st.empty()

    selected_api_integration = api_select.selectbox(
        "Select an API Integration",
        options=git_integrations,
        help="If none of these work, please contact your admin.",
    )

    repo_button = st.empty()
    if repo_button.button("Create Git Repository", key="create_git_repo_btn"):
        try:
            repo_sql = f"""CREATE GIT REPOSITORY IF NOT EXISTS {SOLUTION_NAME}.BASE.EMERGING_SOLUTIONS_TOOLBOX
API_INTEGRATION = "{selected_api_integration}"
ORIGIN = 'https://github.com/Snowflake-Labs/emerging-solutions-toolbox.git'"""

            session.sql(repo_sql).collect()
            add_files_from_git(session)
            api_select.empty()
            repo_button.empty()
            git_repository_deployed = True

        except Exception as e:
            st.warning(
                "Could not create repository.  Try another API Integration or contact an admin.\n  Error: "
                + str(e),
                icon="⚠️",
            )

    confirmation_message()


# Writes a confirmation message if not already written
def confirmation_message():
    global confirm_message_sent

    if database_deployed and files_deployed:
        if not confirm_message_sent:
            status_df = pd.DataFrame(
                [
                    ["Database Deployed", database_deployed],
                    ["Warehouse Deployed", warehouse_deployed],
                    [
                        "Git Repository or Zip Deployed",
                        git_repository_deployed or zip_deployed,
                    ],
                    ["Files Deployed", files_deployed],
                ],
                columns=["Step", "Complete"],
            )

            st.write(status_df)

            st.success(str(SOLUTION_NAME) + " fully deployed!", icon="✅")
            confirm_message_sent = True


# Checks if the git repository has been added
def check_git_repository(session):
    global git_repository_deployed

    # Check for git repository
    git_repository_sql = f"""SHOW GIT REPOSITORIES LIKE 'EMERGING_SOLUTIONS_TOOLBOX' IN DATABASE {SOLUTION_NAME}"""
    git_repository_df = session.sql(git_repository_sql)

    if git_repository_df.count() > 0:
        git_repository_deployed = True


# Checks if the zip file has been uploaded
def check_for_zip(session):
    global zip_deployed

    # Check for zip file
    zip_check_sql = f"""LS @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE} PATTERN='.*emerging-solutions-toolbox-main.zip'"""
    zip_check_df = session.sql(zip_check_sql)

    if zip_check_df.count() > 0:
        zip_deployed = True


# Adds the notebooks and python files to the stage from the git repository
def add_files_from_git(session):
    global files_deployed

    if not files_deployed:
        copy_notebooks_sql = f"""COPY FILES
INTO @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/
FROM @{SOLUTION_NAME}.BASE.EMERGING_SOLUTIONS_TOOLBOX/branches/main/"""

        session.sql(copy_notebooks_sql).collect()
        files_deployed = True


# Adds the notebooks and python files to the stage from a zip file
def add_files_from_zip(session):
    global files_deployed

    if not files_deployed:
        f = session.file.get_stream(
            f"@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main.zip"
        )
        with zipfile.ZipFile(f, "r") as git_zip:
            git_zip.extractall()

        path_list = [
            os.path.join(dirpath, f)
            for (dirpath, dirnames, filenames) in os.walk(
                "emerging-solutions-toolbox-main"
            )
            for f in filenames
        ]

        for path in path_list:
            directory, file_name = os.path.split(path)

            put_result = session.file.put(
                path,
                f"@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/{directory}",
                auto_compress=False,
            )

        files_deployed = True


# Deploys code specific for the solution
def deploy_solution_specific_code(session):
    if files_deployed:
        # Deploy sample data
        sample_data_deployed = False

        sample_data_check_sql = f"""SHOW TABLES LIKE 'DAILY_PARTITIONED_SAMPLE_DATA' IN SCHEMA {SOLUTION_NAME}.BASE"""
        if session.sql(sample_data_check_sql).count() > 0:
            sample_data_deployed = True

        if not sample_data_deployed:
            create_file_format_sql = f"""CREATE OR REPLACE FILE FORMAT {SOLUTION_NAME}.BASE.CSV_FORMAT 
        TYPE = CSV
        FIELD_DELIMITER = ','
        PARSE_HEADER = TRUE
        COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_file_format_sql).collect()

            create_table_sql = f"""CREATE OR REPLACE TABLE {SOLUTION_NAME}.BASE.DAILY_PARTITIONED_SAMPLE_DATA USING TEMPLATE (
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) 
     FROM TABLE (INFER_SCHEMA(
     LOCATION=>'@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/sample_data/daily_partitioned_sample_data.csv',
     FILE_FORMAT=>'{SOLUTION_NAME}.BASE.CSV_FORMAT')))"""
            session.sql(create_table_sql).collect()

            tag_table_sql = f"""ALTER TABLE {SOLUTION_NAME}.BASE.DAILY_PARTITIONED_SAMPLE_DATA SET COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(tag_table_sql).collect()

            load_table_sql = f"""COPY INTO {SOLUTION_NAME}.BASE.DAILY_PARTITIONED_SAMPLE_DATA
    from '@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/sample_data/daily_partitioned_sample_data.csv'
    FILE_FORMAT = (SKIP_HEADER = 1)"""

            session.sql(load_table_sql).collect()

        # Check for notebooks - don't overwrite if present
        notebooks_deployed = False

        notebook_check_sql = f"""LS @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS PATTERN='.*.ipynb'"""
        if session.sql(notebook_check_sql).count() > 0:
            notebooks_deployed = True

        # Copy notebooks and python library files to the right spots
        if not notebooks_deployed:
            copy_notebooks_sql = f"""COPY FILES
    INTO @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS/
    FROM @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/
    PATTERN = '.*.ipynb'"""

            session.sql(copy_notebooks_sql).collect()

            copy_enivronment_yml_sql = f"""COPY FILES
    INTO @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS/
    FROM @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/
    FILES = ('environment.yml')"""

            session.sql(copy_enivronment_yml_sql).collect()

            copy_libraries_sql = f"""COPY FILES
    INTO @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS/forecast_model_builder/
    FROM @{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/forecast_model_builder/
    PATTERN = '.*.py'"""

            session.sql(copy_libraries_sql).collect()

        # Create a stored procedure for creating solution projects
        sp_deploy_sql = f'''CREATE OR REPLACE PROCEDURE {SOLUTION_NAME}.BASE.CREATE_PROJECT(project_name VARCHAR, warehouse VARCHAR)
    RETURNS VARCHAR
    LANGUAGE PYTHON
    RUNTIME_VERSION = '3.11'
    HANDLER = 'create_project'
    PACKAGES = ('snowflake-snowpark-python')
    COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'
    EXECUTE AS CALLER
    AS
$$
def create_project(session, project_name, warehouse):
    create_schema_sql = f"""CREATE SCHEMA IF NOT EXISTS {SOLUTION_NAME}.{{project_name}}
    COMMENT = '{{{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{{{"major":1, "minor":0}}}}, "attributes":{{{{"component":"deployment"}}}}}}}}'"""

    create_eda_notebook_sql = f"""CREATE NOTEBOOK IF NOT EXISTS {SOLUTION_NAME}.{{project_name}}.{{project_name}}__EDA
    FROM '@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS'
    MAIN_FILE = 'eda.ipynb'
    QUERY_WAREHOUSE = {{warehouse}}
    IDLE_AUTO_SHUTDOWN_TIME_SECONDS = 3600
    COMMENT = '{{{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{{{"major":1, "minor":0}}}}, "attributes":{{{{"component":"deployment"}}}}}}}}'"""

    create_modeling_notebook_sql = f"""CREATE NOTEBOOK IF NOT EXISTS {SOLUTION_NAME}.{{project_name}}.{{project_name}}__MODELING
    FROM '@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS'
    MAIN_FILE = 'modeling.ipynb'
    QUERY_WAREHOUSE = {{warehouse}}
    IDLE_AUTO_SHUTDOWN_TIME_SECONDS = 3600
    COMMENT = '{{{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{{{"major":1, "minor":0}}}}, "attributes":{{{{"component":"deployment"}}}}}}}}'"""

    create_inference_notebook_sql = f"""CREATE NOTEBOOK IF NOT EXISTS {SOLUTION_NAME}.{{project_name}}.{{project_name}}__INFERENCE
    FROM '@{SOLUTION_NAME}.BASE.{DEPLOYMENT_STAGE}/NOTEBOOKS'
    MAIN_FILE = 'inference.ipynb'
    QUERY_WAREHOUSE = {{warehouse}}
    IDLE_AUTO_SHUTDOWN_TIME_SECONDS = 3600
    COMMENT = '{{{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{{{"major":1, "minor":0}}}}, "attributes":{{{{"component":"deployment"}}}}}}}}'"""

    session.sql(create_schema_sql).collect()
    session.sql(create_eda_notebook_sql).collect()
    session.sql(create_modeling_notebook_sql).collect()
    session.sql(create_inference_notebook_sql).collect()

    return f"""Project created"""
$$;'''

        session.sql(sp_deploy_sql).collect()


check_permissions(session)
deploy_database(session)
deploy_warehouse(session)
check_stage(session)
deploy_api_integration(session)
deploy_solution_specific_code(session)
confirmation_message()

**Project Creation**

The next step creates a *project*.  A project is new schema with independent set of notebooks and python files, all generated from the base templates. 
 Projects allow for easy isolation of work, while allowing admins to set defaults on the base stages.

- The notebooks are indepenent copies from the base notebook templates
- The python files are copied for each notebook
    - This allows for immediate use of the files without using the imported custom packages UI
    - It does mean that the python files are *independent* copies for each notebook

In [ ]:
# Import python packages

from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Set default value
if "project_name_txt" not in st.session_state:
    st.session_state.project_name_txt = ""


def create_project(project_name):
    if project_name != "BASE":
        with st.spinner():
            session.sql(sql_script).collect()


st.header("New Project")
st.write("Each project gets its own schema and set of notebooks")
project_name = st.text_input("Set your project name", key="project_name_txt")

project_name = project_name.upper().replace(" ", "_")

warehouse_check_sql = """SHOW WAREHOUSES LIKE 'FORECASTING_WH'"""
warehouse_df = session.sql(warehouse_check_sql)

if warehouse_df.count() > 0:
    warehouse = "FORECASTING_WH"
else:
    warehouse_sql = """SELECT CURRENT_WAREHOUSE()"""
    warehouse = session.sql(warehouse_sql).collect()[0][0]

sql_script = f"""CALL FORECAST_MODEL_BUILDER.BASE.CREATE_PROJECT('{project_name}', '{warehouse}')"""

create_col, delete_col = st.columns([6.2, 1])

with create_col:
    create_btn = st.button(
        "Create", key="project_create_btn", on_click=create_project, args={project_name}
    )


with delete_col:
    if st.button("Delete", key="project_delete_btn"):
        if project_name != "BASE":
            with st.spinner():
                delete_sql = (
                    f"""DROP SCHEMA FORECAST_MODEL_BUILDER.{project_name} CASCADE"""
                )
                session.sql(delete_sql).collect()


projects_sql = """SHOW SCHEMAS IN DATABASE FORECAST_MODEL_BUILDER"""
projects_df = session.sql(projects_sql)

project_list = []

for row in projects_df.to_local_iterator():
    if row["name"] not in ["BASE", "INFORMATION_SCHEMA"]:
        project_list.append(row["name"])

st.subheader("Current Projects")
st.dataframe(project_list)

**Next Steps**

Once you've created your project, go to your Notebooks and start with <your project name>__EDA